# Exercise 11

## Car Price Prediction

Predict if the price of a car is low or high

In [64]:
%matplotlib inline
import pandas as pd
import numpy as np

data = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/dataTrain_carListings.zip')
data = data.loc[data['Model'].str.contains('Camry')].drop(['Make', 'State'], axis=1)
data = data.join(pd.get_dummies(data['Model'], prefix='M'))
data['HighPrice'] = (data['Price'] > data['Price'].mean()).astype(int)
data = data.drop(['Model', 'Price'], axis=1)

data.head()

,Year,Mileage,M_Camry,M_Camry4dr,M_CamryBase,M_CamryL,M_CamryLE,M_CamrySE,M_CamryXLE,HighPrice
15,2016,29242,0,0,0,0,1,0,0,1
47,2015,26465,0,0,0,0,1,0,0,1
85,2012,46739,0,1,0,0,0,0,0,1
141,2017,41722,0,0,0,0,0,1,0,1
226,2014,77669,0,0,0,0,0,0,1,0


In [65]:
data.shape

(13150, 10)

In [96]:
y = data['HighPrice']
X = data.drop(['HighPrice'], axis=1)

In [97]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Exercise 11.1

Estimate a Decision Tree Classifier Manually using the code created in the Notebook #13

Evaluate the accuracy on the testing set

In [98]:
max_depth = None
num_pct = 10 #numero de percentiles
max_features = None
min_gain=0.001 #minima ganancia

In [99]:
j = 1
print(X_train.columns[j])

Mileage


In [100]:
splits = np.percentile(X_train.iloc[:, j], np.arange(0, 100, 100.0 / num_pct).tolist())

In [101]:
splits = np.unique(splits)
k = 5

In [102]:
filter_l = X_train.iloc[:, j] < splits[k]

y_l = y_train.loc[filter_l]
y_r = y_train.loc[~filter_l]

In [103]:
def gini(y):
    if y_train.shape[0] == 0:
        return 0
    else:
        return 1 - (y_train.mean()**2 + (1 - y_train.mean())**2)

In [104]:
gini_l = gini(y_l)
gini_l

0.4878029429461156

In [105]:
gini_r = gini(y_r)
gini_r

0.4878029429461156

In [106]:
splits

array([6.000000e+00, 1.603480e+04, 2.324080e+04, 2.987540e+04,
       3.565100e+04, 4.160950e+04, 4.849280e+04, 6.199830e+04,
       8.097420e+04, 1.061537e+05])

In [107]:
def gini_impurity(X_col, y, split):
    "Calculate the gain of an split k on feature j"
    
    filter_l = X_col < split #x solo sirve para sacar el vector booleano que va a definir si pertenece o no al lado izquierdo
    y_l = y.loc[filter_l] #y de la izquierda
    y_r = y.loc[~filter_l]#y de la derecha
    
    n_l = y_l.shape[0]#nodo de la izquierda
    n_r = y_r.shape[0]#nodo de la derecha
    
    gini_y = gini(y) #gini del nodo total, antes de partir
    gini_l = gini(y_l)#gini del lado izquierdo
    gini_r = gini(y_r)#gini del lado derecho
    
    gini_impurity_ = gini_y - (n_l / (n_l + n_r) * gini_l + n_r / (n_l + n_r) * gini_r) #ganancia=gini inicial - suma ponderado del gini lado izq gini del lado der
    
    return gini_impurity_

In [95]:
gini_impurity(X.iloc[:, j], y, splits[k]) 

0.0

In [108]:
def tree_grow(X, y, level=0, min_gain=0.001, max_depth=None, num_pct=10):
    #x completo
    #y completo
    #minima ganancia
    # If only one observation - no se puede partir mas, se devuelve el arbol en donde va
    if X.shape[0] == 1: #crea el directorio con el nivel, split numero observaciones..
        tree = dict(y_pred=y.iloc[:1].values[0], y_prob=0.5, level=level, split=-1, n_samples=1, gain=0)
        return tree
    
    # Calculate the best split
    j, split, gain = best_split(X, y, num_pct)
    
    # save tree and estimate prediction
    y_pred = int(y.mean() >= 0.5) #si mas del 50% es 1 devuelve 1 si no 0
    y_prob = (y.sum() + 1.0) / (y.shape[0] + 2.0)  # Laplace correction - suma 1 al numerador y 2 al denominador que el resultado no tienda a infinito
    
    tree = dict(y_pred=y_pred, y_prob=y_prob, level=level, split=-1, n_samples=X.shape[0], gain=gain)
    
    # Check stooping criteria
    if gain < min_gain:
        return tree
    if max_depth is not None:
        if level >= max_depth:
            return tree   
    
    # No stooping criteria was meet, then continue to create the partition
    filter_l = X.iloc[:, j] < split #se parte x en lado iz y der, por lo que en la sig iteracion se necesita x completo
    X_l, y_l = X.loc[filter_l], y.loc[filter_l]
    X_r, y_r = X.loc[~filter_l], y.loc[~filter_l]
    tree['split'] = [j, split]

    # Next iteration to each split- para cada  particion se vuelve a ejecutar todo
    
    tree['sl'] = tree_grow(X_l, y_l, level + 1, min_gain=min_gain, max_depth=max_depth, num_pct=num_pct)#lado izq
    tree['sr'] = tree_grow(X_r, y_r, level + 1, min_gain=min_gain, max_depth=max_depth, num_pct=num_pct)#lado der
    
    return tree

In [109]:
tree_grow(X, y, level=0, min_gain=0.001, max_depth=1, num_pct=10)

{'y_pred': 1,
 'y_prob': 0.5795316301703163,
 'level': 0,
 'split': -1,
 'n_samples': 13150,
 'gain': 5.551115123125783e-17}

In [110]:
def tree_predict(X, tree, proba=False):#cambia por x_test
    
    predicted = np.ones(X.shape[0])

    # Check if final node
    if tree['split'] == -1:
        if not proba:
            predicted = predicted * tree['y_pred']
        else:
            predicted = predicted * tree['y_prob']
            
    else:
        
        j, split = tree['split']
        filter_l = (X.iloc[:, j] < split)
        X_l = X.loc[filter_l]
        X_r = X.loc[~filter_l]

        if X_l.shape[0] == 0:  # If left node is empty only continue with right
            predicted[~filter_l] = tree_predict(X_r, tree['sr'], proba)
        elif X_r.shape[0] == 0:  # If right node is empty only continue with left
            predicted[filter_l] = tree_predict(X_l, tree['sl'], proba)
        else:
            predicted[filter_l] = tree_predict(X_l, tree['sl'], proba)
            predicted[~filter_l] = tree_predict(X_r, tree['sr'], proba)

    return predicted    

In [111]:
tree = tree_grow(X, y, level=0, min_gain=0.001, max_depth=3, num_pct=10)

In [112]:
tree_predict(X, tree)

array([1., 1., 1., ..., 1., 1., 1.])

In [115]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

NameError: name 'y_pred' is not defined

# Exercise 11.2

Estimate a Bagging of 10 Decision Tree Classifiers Manually using the code created in the Notebook #13

Evaluate the accuracy on the testing set

0.0

(0, 0, 0)

(8810, 0, 8810)

(0.5780930760499432, nan, 0.5780930760499432)

# Exercise 11.3

Implement the variable max_features on the Decision Tree Classifier created in 11.1.

Compare the impact in the results by varing the parameter max_features

Evaluate the accuracy on the testing set

# Exercise 11.4

Estimate a Bagging of 10 Decision Tree Classifiers with `max_features = log(n_features)`

Evaluate the accuracy on the testing set

# Exercise 11.5

Using sklearn, train a RandomForestClassifier

Evaluate the accuracy on the testing set

# Exercise 11.6

Find the best parameters of the RandomForestClassifier (max_depth, max_features, n_estimators)

Evaluate the accuracy on the testing set